#### This is script to query Sentinel-1 scenes through NASA ASF center

input file needs to be prepared in advance in csv format and necessary information is 
* field id (or name)
* geometry
* the season (optional) to be processed
 
output will be a dataframe with informatoin of scene granule that intersect the fields,
    scene location uri at the NASA server, and satellite information
    
url prefix is as follows and curl is used to send string 
url https://api.daac.asf.alaska.edu/services/search/param?

`curl https://api.daac.asf.alaska.edu/services/search/param? platform=Sentinel-1&beamMode=IW&processingLevel=GRD_HS,GRD_HD
&intersectsWith=POLYGON+%28%28-94.01615528012626+42.49834642432543,-93.99159903129991+42.49834642432543,-93.99159903129991+42.52545573340149,-94.01615528012626+42.52545573340149,-94.01615528012626+42.49834642432543%29%29
&start=2019-05-01T00:00:01UTC&end=2019-11-01T00:00:00UTC`

In [ ]:
import datetime
import json
import os
import pandas as pd
import re
import requests
import sys
import time
import urllib.parse
import asf_query as asf
import custom_utils as cutils

from datetime import datetime, timedelta
from dateutil.parser import parse
from requests.auth import HTTPBasicAuth

### define parameters

In [ ]:
URL = "https://api.daac.asf.alaska.edu/services/search/param?"
platform = 'Sentinel-1'
beam_mode='IW'

####  read csv file with field info

In [ ]:
data_dir = '/Users/hojin.kim/Projects/SAR-support/indigo-sar-download/data'

input_csv = 'crf_sar_request_s2Tile.csv'

In [ ]:
df = pd.read_csv(os.path.join(data_dir, input_csv))

In [ ]:
df.head(2)

In [ ]:
df["tillage_date"] = pd.to_datetime(df["tillage_date"]).dt.strftime('%Y-%m-%d')

#### initiate a sesson

In [ ]:
session = requests.session()

#### Let's put it together

In [ ]:
start_time = time.time()
# define a list to store all data frame
append_df =[]

for i, row in df.iterrows():

    field_name = row['field_id']
    base_date = row['tillage_date']
    s2_zoneid = row['S2_Tile_ZoneID']
    print ("query the {}th row for field {}".format(i, field_name))
    geometry_wkt = row['geometry']
    
    # start and end data based on an event
    start, end = cutils.image_query_day_window(base_date, 30)
    
    print (start, end)
    # convert geometry to url format
    url_geometry = asf.wkt_to_url(geometry_wkt)

    # build a query for GRD
    query = asf.build_query(start, end, platform, beam_mode, url_geometry, mapping='S1_GRD')
    
    # get response
    response = session.get(query)

    # output scene boudaries into a geo-dataframe for further steps
    footprint_geodf = asf.found_list_to_df(query)
    footprint_geodf['field_name'] = field_name
    footprint_geodf['s2_zoneid'] = s2_zoneid
    footprint_geodf['field_geometry'] = geometry_wkt

    # append each query
    append_df.append(footprint_geodf)


elaspsed_time_secs = time.time() - start_time
hours, rest = divmod(elaspsed_time_secs,3600)
minutes, seconds = divmod(rest, 60)
print ("Execution took: " , hours, "hrs", minutes, "min", seconds, "secs")

#### stack geodataframe from the list

In [ ]:
stack_gdf = pd.concat(append_df, axis=0)

In [ ]:
stack_gdf.info()

In [ ]:
outfbase = input_csv.split('.')[0]
stack_gdf.to_csv(os.path.join(data_dir, 
                "{}_s1_acquistion.csv".format(outfbase, index=False)))

#### drop duplicate granule id